In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from tpot import TPOTRegressor
from sklearn.metrics import make_scorer
import matplotlib.pyplot as plt
from featuresv2 import targets, metadata
from position_group_features import features
import json
import requests
%matplotlib inline

## Version 4 Model

In [2]:
data = pd.read_csv('outputs/model_training_data.csv')

In [3]:
data2023 = data[data['round.year'] != 2023]

In [4]:
round_matchups = [
    ('Sydney Swans','Melbourne'),
    ('Brisbane Lions','Carlton'),
    ('Gold Coast Suns','Richmond'),
    ('GWS Giants','Collingwood')]

In [5]:
round1 = [('Carlton', 'Richmond'),
('Collingwood','Sydney Swans'),
('Essendon','Hawthorn'),
('GWS Giants','North Melbourne'),
('Geelong Cats','St Kilda'),
('Gold Coast Suns','Adelaide Crows'),
('Melbourne','Western Bulldogs'),
('Port Adelaide','West Coast Eagles'),
('Fremantle','Brisbane Lions')]

In [6]:
def generate_test_train_split(data, features, target, test_size=0.1):
    X = data[features]
    y = data[target]

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    
    return {"x_train":x_train,"x_test":x_test,"y_train":y_train,"y_test":y_test}

In [7]:
def tpot_pipeline(data, features, target, size=20, scoring='r2'):
    data_ = generate_test_train_split(data, features, target)

    my_scorer = make_scorer(mean_squared_error, greater_is_better=False)

    pipeline_optimizer = TPOTRegressor(generations=5, population_size=size, verbosity=2, scoring=scoring)

    pipeline_optimizer.fit(data_['x_train'], data_['y_train'])
    
    print(pipeline_optimizer.score(data_['x_test'], data_['y_test']))
    
    return pipeline_optimizer

In [8]:
def return_stats_input(data, home_team, away_team, model_inputs, games=5):
    '''Take in player_stats and returns aggregated team stats for previous home/away team matchups'''
    
#     if 'team.name' not in model_inputs:
#         model_inputs.append('team.name')
        
    match_ups = data[(data['match.homeTeam.name']==home_team) & (data['match.awayTeam.name']==away_team)]
    weights = range(1,len(match_ups)+1)

    stats = match_ups[model_inputs+['match.homeTeam.name']].groupby('match.homeTeam.name').apply(lambda x: np.average(x, weights=weights, axis=0))
    
    return pd.DataFrame(dict(zip(model_inputs,stats[0])),index=[0])

In [9]:
def generate_margin_tear_off(model, round_matchups, team_stats, model_inputs):
    print('-------------------' + str(model.fitted_pipeline_[0]).replace('()','') + ' TEAR-OFF-------------------')
    for tup in round_matchups:
        predict = model.predict(return_stats_input(team_stats, tup[0], tup[1], model_inputs))[0]
        if predict < 0:
            winner = tup[1]
        else:
            winner = tup[0]
        print("------------------------------------------------")
        print("Home: " + tup[0])
        print("Away: " + tup[1])
        print("Winner: " + winner + "\t       " + "Margin: " + str(abs(round(predict))))

In [10]:
def generate_score_tear_off(home_model, away_model, round_matchups, team_stats, model_inputs):
   # print('-------------------' + str(model.fitted_pipeline_[0]).replace('()','') + ' TEAR-OFF-------------------')
    preds = []
    for tup in round_matchups:
        try:
            h_predict = home_model.predict(return_stats_input(team_stats, tup[0], tup[1], model_inputs))[0]
            a_predict = away_model.predict(return_stats_input(team_stats, tup[0], tup[1], model_inputs))[0]
            if h_predict < a_predict:
                winner = tup[1]
                margin = round(a_predict) - round(h_predict)
            else:
                winner = tup[0]
                margin = round(h_predict) - round(a_predict)
            paddingh = 25 - len("Home: " + tup[0])
            paddinga = 25 - len("Away: " + tup[1])
            print("------------------------------------------------")
            print("Home: " + tup[0] + " "*paddingh + str(abs(round(h_predict))))
            print("Away: " + tup[1] + " "*paddinga + str(abs(round(a_predict))))
            print("Winner: " + winner + " by "+ str(margin) + " points")
            print("Total Score: " + str(round(h_predict) + round(a_predict)))
            preds.append((h_predict, a_predict, margin))
        except:
            print("No prior game")
            preds.append((0, 0, 0))
    return preds

In [11]:
home_model = tpot_pipeline(data, features, targets[0], size=10)

Optimization Progress:   0%|          | 0/60 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9397505031560384

Generation 2 - Current best internal CV score: 0.9397505031560384

Generation 3 - Current best internal CV score: 0.9397505031560384

Generation 4 - Current best internal CV score: 0.9397505031560384

Generation 5 - Current best internal CV score: 0.9434867270396335

Best pipeline: ElasticNetCV(MinMaxScaler(input_matrix), l1_ratio=0.8, tol=1e-05)
0.9197378733312305


In [12]:
away_model = tpot_pipeline(data, features, targets[1], size=10)

Optimization Progress:   0%|          | 0/60 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9505816855770817

Generation 2 - Current best internal CV score: 0.9505816855770817

Generation 3 - Current best internal CV score: 0.9505816855770817

Generation 4 - Current best internal CV score: 0.9505816855770817

Generation 5 - Current best internal CV score: 0.9505816855770817

Best pipeline: ElasticNetCV(StandardScaler(input_matrix), l1_ratio=1.0, tol=0.01)
0.9391635032790449


In [13]:
# margin_model = tpot_pipeline(data, features, targets[2], size=10)

In [14]:
#generate_score_tear_off(home_model, away_model, round1, data, features)

In [15]:
# print(return_stats_inputv3(data, home_team, away_team, features))
# generate_margin_tear_off(margin_model, round1, data, features)
# data[(data['match.homeTeam.name']==home_team) & (data['match.awayTeam.name']==away_team)][features].head()

In [16]:
fix_data_2024 = json.loads(requests.get('https://fixturedownload.com/feed/json/afl-2024').content)

fix_data_2023 = json.loads(requests.get('https://fixturedownload.com/feed/json/afl-2023').content)

fixtures2024 = pd.DataFrame(fix_data_2024)
games2024 = list(zip(fixtures2024.HomeTeam, fixtures2024.AwayTeam))

fixtures2023 = pd.DataFrame(fix_data_2023)
games2023 = list(zip(fixtures2023.HomeTeam, fixtures2023.AwayTeam))

predicted_season = generate_score_tear_off(home_model, away_model, games2024, data, features)

fixtures2024['predicted_home'] = [round(i[0]) for i in predicted_season]
fixtures2024['predicted_away'] = [round(i[1]) for i in predicted_season]

pred_winner = []
winner = []
for i, row in fixtures2024.iterrows():

    if row[7] > row[8]:
        winner.append(row[4])
    elif row[8] > row[7]:
        winner.append(row[5])
    else:
        winner.append('Draw')


    if row[9] > row[10]:
        pred_winner.append(row[4])
    elif row[10] > row[9]:
        pred_winner.append(row[5])
    else:
        pred_winner.append('Draw')

fixtures2024['predicted_winner'] = pred_winner
fixtures2024['winner'] = winner
fixtures2024['predicted_margin'] = fixtures2024['predicted_home'] - fixtures2024['predicted_away']
fixtures2024['actual_margin'] = fixtures2024['HomeTeamScore'] - fixtures2024['AwayTeamScore']
fixtures2024['correct'] = [1 if row[11]==row[12] else 0 for i,row in fixtures2024.iterrows()]
# fixtures2023['odds'] = [2.3 if abs(row[13]) < 40 else 2.6 for i,row in fixtures2023.iterrows()]


fixtures2024.to_csv('outputs/predictions_and_results_2024_.csv', index=False)

------------------------------------------------
Home: Sydney Swans       75
Away: Melbourne          68
Winner: Sydney Swans by 7 points
Total Score: 143
------------------------------------------------
Home: Brisbane Lions     94
Away: Carlton            74
Winner: Brisbane Lions by 20 points
Total Score: 168


C:\Users\SeanW\AppData\Local\Temp\ipykernel_12728\905886690.py:10: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  stats = match_ups[model_inputs+['match.homeTeam.name']].groupby('match.homeTeam.name').apply(lambda x: np.average(x, weights=weights, axis=0))


No prior game


C:\Users\SeanW\AppData\Local\Temp\ipykernel_12728\905886690.py:10: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  stats = match_ups[model_inputs+['match.homeTeam.name']].groupby('match.homeTeam.name').apply(lambda x: np.average(x, weights=weights, axis=0))


No prior game
------------------------------------------------
Home: Carlton            83
Away: Richmond           86
Winner: Richmond by 3 points
Total Score: 169
------------------------------------------------
Home: Collingwood        66
Away: Sydney Swans       75
Winner: Sydney Swans by 9 points
Total Score: 141
------------------------------------------------
Home: Essendon           100
Away: Hawthorn           83
Winner: Essendon by 17 points
Total Score: 183


...

No prior game
------------------------------------------------
Home: Carlton            75
Away: St Kilda           79
Winner: St Kilda by 4 points
Total Score: 154
------------------------------------------------
Home: Fremantle          71
Away: Port Adelaide      80
Winner: Port Adelaide by 9 points
Total Score: 151
------------------------------------------------
Home: Port Adelaide      83
Away: Geelong Cats       91
Winner: Geelong Cats by 8 points
Total Score: 174
------------------------------------------------
Home: Western Bulldogs   81
Away: Hawthorn           93
Winner: Hawthorn by 12 points
Total Score: 174


C:\Users\SeanW\AppData\Local\Temp\ipykernel_12728\905886690.py:10: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  stats = match_ups[model_inputs+['match.homeTeam.name']].groupby('match.homeTeam.name').apply(lambda x: np.average(x, weights=weights, axis=0))


No prior game
------------------------------------------------
Home: Brisbane Lions     94
Away: Carlton            74
Winner: Brisbane Lions by 20 points
Total Score: 168
------------------------------------------------
Home: Port Adelaide      87
Away: Hawthorn           84
Winner: Port Adelaide by 3 points
Total Score: 171
No prior game


C:\Users\SeanW\AppData\Local\Temp\ipykernel_12728\905886690.py:10: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  stats = match_ups[model_inputs+['match.homeTeam.name']].groupby('match.homeTeam.name').apply(lambda x: np.average(x, weights=weights, axis=0))


------------------------------------------------
Home: Sydney Swans       82
Away: Port Adelaide      73
Winner: Sydney Swans by 9 points
Total Score: 155
------------------------------------------------
Home: Geelong Cats       96
Away: Brisbane Lions     73
Winner: Geelong Cats by 23 points
Total Score: 169
------------------------------------------------
Home: Sydney Swans       82
Away: Brisbane Lions     88
Winner: Brisbane Lions by 6 points
Total Score: 170


In [17]:
# backtest_data = pd.read_csv('outputs/predictions_and_results_2024.csv')
# backtest_data.columns

In [18]:
# profit_curve_no_geelong = []
# bankroll = 100
# stake = 0
# profit = 0
# for i, row in backtest_data.iterrows():
    
#     if (backtest_data['predicted_winner'][i] != 'Geelong Cats') and (backtest_data['predicted_winner'][i] != 'Richmond'):
#         stake = -1 * 0.1 * bankroll
#         profit = -stake * backtest_data['odds'][i] * backtest_data['win '][i]
        
#         bankroll+=stake+profit

#         profit_curve_no_geelong.append(bankroll)
#     else:
#         bankroll+=0

#         profit_curve_no_geelong.append(bankroll)

In [19]:
# profit_curve = []
# bankroll = 100
# stake = 0
# profit = 0
# for i, row in backtest_data.iterrows():
    
#     stake = -1 * 0.1 * bankroll
#     profit = -stake * backtest_data['odds'][i] * backtest_data['win '][i]

#     bankroll+=stake+profit

#     profit_curve.append(bankroll)

In [20]:
# plt.plot(profit_curve, color='b')
# plt.plot(profit_curve_no_geelong, color='r')